<a href="https://colab.research.google.com/github/Hironobu-Kawaguchi/kmnist/blob/master/colab_kmnist_pytorch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as dataset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        # x = F.relu(x)   # add
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [0]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=True,
                        help='For Saving the current Model')
    # args = parser.parse_args()
    args = parser.parse_args(args=[])
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.KMNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.KMNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "kmnist_cnn.pt")


In [8]:
main()

0it [00:00, ?it/s]

18169856it [00:02, 6514845.44it/s]                              


Extracting ../data/KMNIST/raw/train-images-idx3-ubyte.gz to ../data/KMNIST/raw


  0%|          | 0/29497 [00:00<?, ?it/s]

32768it [00:00, 117698.45it/s]           
  0%|          | 0/3041136 [00:00<?, ?it/s]

Extracting ../data/KMNIST/raw/train-labels-idx1-ubyte.gz to ../data/KMNIST/raw


3047424it [00:01, 2659425.79it/s]                             
0it [00:00, ?it/s]

Extracting ../data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/KMNIST/raw


8192it [00:00, 44970.30it/s]            


Extracting ../data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/KMNIST/raw
Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.336921
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.404426
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.940515
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.745151
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.846319
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.605437
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.807723
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.565172
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.442035
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.510495
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.587230
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.327208
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.607165
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.471731
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.453331
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.450253
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.641903
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.280010
Train Epoch: 1 [11520/60000 (19

In [10]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.323755
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.718463
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.157521
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.011413
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.853044
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.682405
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.138477
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.748775
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.466604
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.580805
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.649432
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.359211
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.561017
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.378980
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.438140
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.556191
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.514167
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.275147
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.443455
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.415373
Train Epoch: 1 [12800/60000 (

In [15]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.336921
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.404953
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.960047
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.768878
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.820041
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.547985
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.918830
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.577455
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.466553
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.523246
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.561410
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.241484
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.557434
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.487450
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.356376
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.428395
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.549146
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.259448
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.359254
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.433968
Train Epoch: 1 [12800/60000 (

In [16]:
torch.__version__

'1.3.1'

In [19]:
import torchvision
torchvision.__version__

'0.4.2'

In [20]:
np.__version__

'1.17.4'

In [21]:
import sys
sys.version

'3.6.9 (default, Nov  7 2019, 10:44:02) \n[GCC 8.3.0]'